In [2]:
import os
from glob import glob 
import pandas as pd
from tqdm import tqdm
from natsort import natsorted
import cv2
import random 
import numpy as np
import sys
import random
sys.path.append("/home/dais01/VirtualFlaw/RT_Project")
from modules import flaw
import ray
ray.init(num_cpus=70, ignore_reinit_error=True)
PO_DIFF = flaw.PO_DIFF

#warning 지우기
import warnings
warnings.filterwarnings(action='ignore')


Flaw Config Version: 1.0.1


2023-07-06 13:49:40,953	INFO worker.py:1625 -- Started a local Ray instance.


In [12]:
png_images = glob("/home/dais01/VF_MASTER/Generated_Flaw/CT/**/*.png", recursive=True)
print(len(png_images))
png_images = glob("/home/dais01/VF_MASTER/Generated_Flaw/Leftover/**/*.png", recursive=True)
print(len(png_images))
png_images = glob("/home/dais01/VF_MASTER/Generated_Flaw/PO/**/*.png", recursive=True)
print(len(png_images))
png_images = glob("/home/dais01/VF_MASTER/Generated_Flaw/Scratch/**/*.png", recursive=True)
print(len(png_images))


32535
32559
31266
32559


In [4]:
image_info = pd.read_csv("/home/dais01/VF_MASTER/Dataset_ALL/RT_Image_Info.csv")
print(image_info.columns)
image_info = image_info[image_info['NDT_result'] == 'Accept']
image_info = image_info[image_info['EndTab'] == 'N']
image_info = image_info[image_info['Rework'] == 'N']
thickness_list = image_info["thickness"].unique()
thickness_list = natsorted(thickness_list)
thickness_list = thickness_list[:12]
print(thickness_list)

    

Index(['image_path', 'image_name', 'category', 'thickness', 'NDT_result',
       'Type', 'EndTab', 'Rework'],
      dtype='object')
['7.92mm', '8.74mm', '9.0mm', '9.5mm', '9.53mm', '10.31mm', '11.13mm', '12.0mm', '12.7mm', '14.27mm', '15.09mm', '15.88mm']


In [10]:
#두께별로 3천개씩 추출
df_concat = pd.DataFrame()
for thickness in thickness_list:
    df_temp = image_info[image_info["thickness"] == thickness]
    try:
        df_temp = df_temp.sample(n=5000)
    except ValueError:
        df_temp = df_temp.sample(n=len(df_temp))
    df_concat = pd.concat([df_concat, df_temp])
    
df = df_concat
print(df.shape) 


path_list = []
for i in range(len(df)):
    image = df.iloc[i]["image_name"]
    category = df.iloc[i]["category"]
    thickness = df.iloc[i]["thickness"].replace("mm", "")
    file_path = df.iloc[i]["image_path"]
    path_list.append(file_path)
    
    
#path_list에서 300개 추출
path_list = random.sample(path_list, 1000)



(38991, 8)


In [11]:
PO_list = glob("/home/dais01/VF_MASTER/Data/PO/Deformed_Flaw/*.npy")
Scratch_list = glob("/home/dais01/VF_MASTER/Data/Scratch/Deformed_Flaw/*.npy")
Leftover_list = glob("/home/dais01/VF_MASTER/Data/Leftover/Deformed_Flaw/*.npy")
CT_list = glob("/home/dais01/VF_MASTER/Data/CT/Deformed_Flaw/*.npy")


@ray.remote
def generate_virtual_flaw(image_path, padding = -40, fade = 50, flaw_type = "PO"):
    image_name = image_path.split("/")[-1]
    print(image_name)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) # 이미지를 불러옴.
    origin_image = image.copy() # 원본 이미지 저장. 나중에 비교해서 보여주려고.
    image = np.float32(image) # 이미지를 float32로 변환. 결함을 합성할 때 정수가 아니라 실수로 계산하고 나중에 uint8로 변환함.
    calc_image = image.copy() # 용접부 찾을 이미지.

    # Masking
    PO_mask_image = np.zeros_like(image) # PO 결함들을 합성할 위치를 나타내는 마스크.
    Scratch_Leftover_mask_image = np.ones_like(image)
    CT_mask_image = np.zeros_like(image)

    # 용접부 Detection
    calc_image = cv2.rotate(calc_image, cv2.ROTATE_90_CLOCKWISE)
    calc_image = cv2.normalize(calc_image, None, 0, 1000, cv2.NORM_MINMAX, cv2.CV_32F)
    calc_image = cv2.GaussianBlur(calc_image, (31,31), 0)
    calc_image = cv2.resize(calc_image, (512, 512))
    calc_image = np.array(calc_image, dtype=np.float32)
    try:
        for i in range(9):
            calc_image = np.split(calc_image, 2, axis=0)
            calc_image = np.add(calc_image[0], calc_image[1])

        calc_image = np.gradient(np.squeeze(calc_image))
        y1, y2 = 1256 - int(np.argmax(calc_image)* 1256 / 512), 1256 - int(np.argmin(calc_image)* 1256 / 512)
        if y1 < 100 or y2 > 1056:
            raise Exception("y1 or y2 is out of range")
        
        y1 = y1 - padding
        y2 = y2 + padding
        # Masking -> 용접부는 흰색
        PO_mask_image[y1:y2, :] = 1
        # 용접부의 위아래 padding 부분에는 점점 0으로 줄어들게 만들기
        PO_mask_image[y1-fade:y1, :] = np.repeat(np.linspace(0, 1, fade)[:, None], PO_mask_image.shape[1], axis=1)
        PO_mask_image[y2:y2+fade, :] = np.repeat(np.linspace(1, 0, fade)[:, None], PO_mask_image.shape[1], axis=1)
        Scratch_Leftover_mask_image = Scratch_Leftover_mask_image - PO_mask_image
        # Scratch_Leftover_mask_image는 PO_mask_image의 반전
        
        
        CT_mask_image[y1-60:y2+60, :] = 1 # CT는 모재부에도 침범하는 결함을 합성할 수 있도록.
        CT_mask_image[y1-60-fade:y1-60, :] = np.repeat(np.linspace(0, 1, fade)[:, None], CT_mask_image.shape[1], axis=1)
        CT_mask_image[y2+60:y2+60+fade, :] = np.repeat(np.linspace(1, 0, fade)[:, None], CT_mask_image.shape[1], axis=1)

        temp_image = np.zeros_like(image)
        #세로로 이어붙인다. 임시 이미지에 결함을 합성하고 나중에 다시 잘라내고 PO_mask_image를 곱한다.
        temp_image = np.concatenate([temp_image, PO_mask_image, temp_image,], axis=0)
        flaw_image = np.zeros_like(temp_image)
        y1, y2 = y1 + 1256, y2 + 1256
        if flaw_type == "Random":
            flaw_type = np.random.choice(["PO", "CT", "Scratch", "Leftover"])
        
    
        if flaw_type == "PO":
            random_try = np.random.randint(3, 6)
            for _ in range(random_try):
                random_flaw = np.random.choice(PO_list)
                random_flaw = np.load(random_flaw)
                random_flaw = np.asarray(random_flaw, dtype=np.float32)
                #flaw_image의 랜덤한 위치에 random_flaw를 넣기
                x = np.random.randint(0, image.shape[1] - random_flaw.shape[1])
                y = np.random.randint(y1, y2 - random_flaw.shape[0])
                
                flaw_image[y:y+random_flaw.shape[0], x:x+random_flaw.shape[1]] += random_flaw 
                
            flaw_image = flaw_image[1256:1256*2, :]
            flaw_image = flaw_image * PO_mask_image
            # 원본이미지와 같은 사이즈의 검정색 이미지 2장을 준비함
            # 1장(flaw_image)은 결함만 두두두두, 1장은 용접부 경계선 스무딩한 것 -> 곱함 -> 용접부 경계선에 있는 결함은 흐려짐
            # 정상이미지에 더함
            

        if flaw_type == "CT":
            random_flaw = np.random.choice(CT_list)
            random_flaw = np.load(random_flaw)
            random_flaw = np.asarray(random_flaw, dtype=np.float32)
            #flaw_image의 랜덤한 위치에 random_flaw를 넣기
            x = np.random.randint(0, image.shape[1] - random_flaw.shape[1])
            y = np.random.randint(y1-random_flaw.shape[0], y2)
            flaw_image[y:y+random_flaw.shape[0], x:x+random_flaw.shape[1]] += random_flaw * 1
                
            flaw_image = flaw_image[1256:1256*2, :]
            flaw_image = flaw_image * CT_mask_image
                
        elif flaw_type == "Scratch":
            random_try = np.random.randint(2, 3)
            for _ in range(random_try):
                random_flaw = np.random.choice(Scratch_list)
                random_flaw = np.load(random_flaw)
                random_flaw = np.asarray(random_flaw, dtype=np.float32)
                #flaw_image의 랜덤한 위치에 random_flaw를 넣기
                x = np.random.randint(0, image.shape[1] - random_flaw.shape[1])
                y_top = np.random.randint(y2, y2 + 100)
                y_bottom = np.random.randint(y1 - 100, y1 )
                y = np.random.choice([y_top, y_bottom])
                if y == y_bottom:
                    flaw_image[y-random_flaw.shape[0]:y, x:x+random_flaw.shape[1]] += random_flaw * 1.5
                    
                elif y == y_top:
                    flaw_image[y:y+random_flaw.shape[0], x:x+random_flaw.shape[1]] += random_flaw * 1.5
                
            flaw_image = flaw_image[1256:1256*2, :]
            flaw_image = flaw_image * Scratch_Leftover_mask_image
            
        elif flaw_type == "Leftover":
            random_try = np.random.randint(3, 4)
            for _ in range(random_try):
                random_flaw = np.random.choice(Leftover_list)
                random_flaw = np.load(random_flaw)
                random_flaw = np.asarray(random_flaw, dtype=np.float32)
                #flaw_image의 랜덤한 위치에 random_flaw를 넣기
                x = np.random.randint(0, image.shape[1] - random_flaw.shape[1])
                y_top = np.random.randint(y2, y2 + 600)
                y_bottom = np.random.randint(y1 - 600, y1 )
                y = np.random.choice([y_top, y_bottom])
                if y == y_bottom:
                    flaw_image[y-random_flaw.shape[0]:y, x:x+random_flaw.shape[1]] += random_flaw * 1
                elif y == y_top:
                    flaw_image[y:y+random_flaw.shape[0], x:x+random_flaw.shape[1]] += random_flaw * 1
            
            flaw_image = flaw_image[1256:1256*2, :]
            flaw_image = flaw_image * Scratch_Leftover_mask_image
        image += flaw_image
        #image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
        
        save_path = f"/home/dais01/VF_MASTER/Generated_Flaw/{flaw_type}"
        os.makedirs(save_path + "/Accept", exist_ok=True)
        os.makedirs(save_path + "/Reject", exist_ok=True)
        os.makedirs(save_path + "/Diff", exist_ok=True)
        
        #합성된 이미지
        cv2.imwrite(save_path + "/Reject/" + image_name, image, [cv2.IMWRITE_PNG_COMPRESSION, 0])
        #차이 이미지. flaw_image에서 0이 아닌 픽셀은 모두 255로 만들어서 저장
        diff = np.zeros_like(image)
        diff = np.where(flaw_image != 0, 255, 0)
        cv2.imwrite(save_path + "/Diff/" + image_name.replace(".png", "_diff.png"), diff, [cv2.IMWRITE_PNG_COMPRESSION, 0])
        #원본이미지
        #origin_image = cv2.normalize(origin_image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
        cv2.imwrite(save_path + "/Accept/" + image_name, origin_image, [cv2.IMWRITE_PNG_COMPRESSION, 0])
        
    except:
        pass
        
    
        
            
        
        
for flaw_type in ["PO", "CT", "Scratch", "Leftover"]:
    ray.get([generate_virtual_flaw.remote(path_list[i], flaw_type = flaw_type) for i in tqdm(range(len(path_list)))])

         
        
        
        

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:00<00:00, 7047.14it/s]


(generate_virtual_flaw pid=59587) 22G009-03-006-1st-1-(1)-24.png [repeated 436x across cluster]


100%|██████████| 1000/1000 [00:00<00:00, 4281.72it/s]


(generate_virtual_flaw pid=59632) 22G009-25-004-1st-1-(1)-38.png [repeated 1167x across cluster]


100%|██████████| 1000/1000 [00:00<00:00, 4264.94it/s]


(generate_virtual_flaw pid=59604) 22I018-05-004-1st-1-(2)-42.png [repeated 1303x across cluster]


100%|██████████| 1000/1000 [00:00<00:00, 4762.55it/s]


(generate_virtual_flaw pid=59636) 22H007-01-001-1st-1-(1)-34.png [repeated 1315x across cluster]
